Taken from: https://www.kaggle.com/adityadesai13/used-car-dataset-ford-and-mercedes

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install scikit-optimize

     |████████████████████████████████| 102kB 3.2MB/s 


In [4]:
import pandas as pd
import sys
sys.path.insert(0,'/content/drive/My Drive/Datasets_SVR/Library')
from subsamplesvr import svr_subsample_bayes_train
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
from sklearn.svm import SVR
import numpy as np
import time
import math
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV

In [5]:
audi_data=pd.read_csv("/content/drive/My Drive/Datasets_SVR/Data/Used Cars Datasets/audi.csv")
bmw_data=pd.read_csv("/content/drive/My Drive/Datasets_SVR/Data/Used Cars Datasets/bmw.csv")
ford_data=pd.read_csv("/content/drive/My Drive/Datasets_SVR/Data/Used Cars Datasets/ford.csv")
hyundi_data=pd.read_csv("/content/drive/My Drive/Datasets_SVR/Data/Used Cars Datasets/hyundi.csv")
merc_data=pd.read_csv("/content/drive/My Drive/Datasets_SVR/Data/Used Cars Datasets/merc.csv")
skoda_data=pd.read_csv("/content/drive/My Drive/Datasets_SVR/Data/Used Cars Datasets/skoda.csv")
toyota_data=pd.read_csv("/content/drive/My Drive/Datasets_SVR/Data/Used Cars Datasets/toyota.csv")
vauxhall_data=pd.read_csv("/content/drive/My Drive/Datasets_SVR/Data/Used Cars Datasets/vauxhall.csv")
vw_data=pd.read_csv("/content/drive/My Drive/Datasets_SVR/Data/Used Cars Datasets/vw.csv")


In [6]:
print(audi_data.info())
display(audi_data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10668 entries, 0 to 10667
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         10668 non-null  object 
 1   year          10668 non-null  int64  
 2   price         10668 non-null  int64  
 3   transmission  10668 non-null  object 
 4   mileage       10668 non-null  int64  
 5   fuelType      10668 non-null  object 
 6   tax           10668 non-null  int64  
 7   mpg           10668 non-null  float64
 8   engineSize    10668 non-null  float64
dtypes: float64(2), int64(4), object(3)
memory usage: 750.2+ KB
None


,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,A1,2017,12500,Manual,15735,Petrol,150,55.4,1.4
1,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0
2,A1,2016,11000,Manual,29946,Petrol,30,55.4,1.4
3,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0
4,A3,2019,17300,Manual,1998,Petrol,145,49.6,1.0


In [7]:
print(bmw_data.info())
display(bmw_data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10781 entries, 0 to 10780
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         10781 non-null  object 
 1   year          10781 non-null  int64  
 2   price         10781 non-null  int64  
 3   transmission  10781 non-null  object 
 4   mileage       10781 non-null  int64  
 5   fuelType      10781 non-null  object 
 6   tax           10781 non-null  int64  
 7   mpg           10781 non-null  float64
 8   engineSize    10781 non-null  float64
dtypes: float64(2), int64(4), object(3)
memory usage: 758.2+ KB
None


,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,5 Series,2014,11200,Automatic,67068,Diesel,125,57.6,2.0
1,6 Series,2018,27000,Automatic,14827,Petrol,145,42.8,2.0
2,5 Series,2016,16000,Automatic,62794,Diesel,160,51.4,3.0
3,1 Series,2017,12750,Automatic,26676,Diesel,145,72.4,1.5
4,7 Series,2014,14500,Automatic,39554,Diesel,160,50.4,3.0


In [8]:
print(ford_data.info())
display(ford_data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17965 entries, 0 to 17964
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         17965 non-null  object 
 1   year          17965 non-null  int64  
 2   price         17965 non-null  int64  
 3   transmission  17965 non-null  object 
 4   mileage       17965 non-null  int64  
 5   fuelType      17965 non-null  object 
 6   tax           17965 non-null  int64  
 7   mpg           17965 non-null  float64
 8   engineSize    17965 non-null  float64
dtypes: float64(2), int64(4), object(3)
memory usage: 1.2+ MB
None


,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,Fiesta,2017,12000,Automatic,15944,Petrol,150,57.7,1.0
1,Focus,2018,14000,Manual,9083,Petrol,150,57.7,1.0
2,Focus,2017,13000,Manual,12456,Petrol,150,57.7,1.0
3,Fiesta,2019,17500,Manual,10460,Petrol,145,40.3,1.5
4,Fiesta,2019,16500,Automatic,1482,Petrol,145,48.7,1.0


In [9]:
print(hyundi_data.info())
display(hyundi_data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4860 entries, 0 to 4859
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         4860 non-null   object 
 1   year          4860 non-null   int64  
 2   price         4860 non-null   int64  
 3   transmission  4860 non-null   object 
 4   mileage       4860 non-null   int64  
 5   fuelType      4860 non-null   object 
 6   tax(£)        4860 non-null   int64  
 7   mpg           4860 non-null   float64
 8   engineSize    4860 non-null   float64
dtypes: float64(2), int64(4), object(3)
memory usage: 341.8+ KB
None


,model,year,price,transmission,mileage,fuelType,tax(£),mpg,engineSize
0,I20,2017,7999,Manual,17307,Petrol,145,58.9,1.2
1,Tucson,2016,14499,Automatic,25233,Diesel,235,43.5,2.0
2,Tucson,2016,11399,Manual,37877,Diesel,30,61.7,1.7
3,I10,2016,6499,Manual,23789,Petrol,20,60.1,1.0
4,IX35,2015,10199,Manual,33177,Diesel,160,51.4,2.0


In [10]:
hyundi_data.rename(columns={'tax(£)':'tax'}, inplace=True)
hyundi_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4860 entries, 0 to 4859
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         4860 non-null   object 
 1   year          4860 non-null   int64  
 2   price         4860 non-null   int64  
 3   transmission  4860 non-null   object 
 4   mileage       4860 non-null   int64  
 5   fuelType      4860 non-null   object 
 6   tax           4860 non-null   int64  
 7   mpg           4860 non-null   float64
 8   engineSize    4860 non-null   float64
dtypes: float64(2), int64(4), object(3)
memory usage: 341.8+ KB


In [11]:
print(merc_data.info())
display(merc_data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13119 entries, 0 to 13118
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         13119 non-null  object 
 1   year          13119 non-null  int64  
 2   price         13119 non-null  int64  
 3   transmission  13119 non-null  object 
 4   mileage       13119 non-null  int64  
 5   fuelType      13119 non-null  object 
 6   tax           13119 non-null  int64  
 7   mpg           13119 non-null  float64
 8   engineSize    13119 non-null  float64
dtypes: float64(2), int64(4), object(3)
memory usage: 922.6+ KB
None


,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,SLK,2005,5200,Automatic,63000,Petrol,325,32.1,1.8
1,S Class,2017,34948,Automatic,27000,Hybrid,20,61.4,2.1
2,SL CLASS,2016,49948,Automatic,6200,Petrol,555,28.0,5.5
3,G Class,2016,61948,Automatic,16000,Petrol,325,30.4,4.0
4,G Class,2016,73948,Automatic,4000,Petrol,325,30.1,4.0


In [12]:
print(skoda_data.info())
display(skoda_data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6267 entries, 0 to 6266
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         6267 non-null   object 
 1   year          6267 non-null   int64  
 2   price         6267 non-null   int64  
 3   transmission  6267 non-null   object 
 4   mileage       6267 non-null   int64  
 5   fuelType      6267 non-null   object 
 6   tax           6267 non-null   int64  
 7   mpg           6267 non-null   float64
 8   engineSize    6267 non-null   float64
dtypes: float64(2), int64(4), object(3)
memory usage: 440.8+ KB
None


,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,Octavia,2017,10550,Manual,25250,Petrol,150,54.3,1.4
1,Citigo,2018,8200,Manual,1264,Petrol,145,67.3,1.0
2,Octavia,2019,15650,Automatic,6825,Diesel,145,67.3,2.0
3,Yeti Outdoor,2015,14000,Automatic,28431,Diesel,165,51.4,2.0
4,Superb,2019,18350,Manual,10912,Petrol,150,40.9,1.5


In [13]:
print(toyota_data.info())
display(toyota_data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6738 entries, 0 to 6737
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         6738 non-null   object 
 1   year          6738 non-null   int64  
 2   price         6738 non-null   int64  
 3   transmission  6738 non-null   object 
 4   mileage       6738 non-null   int64  
 5   fuelType      6738 non-null   object 
 6   tax           6738 non-null   int64  
 7   mpg           6738 non-null   float64
 8   engineSize    6738 non-null   float64
dtypes: float64(2), int64(4), object(3)
memory usage: 473.9+ KB
None


,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,GT86,2016,16000,Manual,24089,Petrol,265,36.2,2.0
1,GT86,2017,15995,Manual,18615,Petrol,145,36.2,2.0
2,GT86,2015,13998,Manual,27469,Petrol,265,36.2,2.0
3,GT86,2017,18998,Manual,14736,Petrol,150,36.2,2.0
4,GT86,2017,17498,Manual,36284,Petrol,145,36.2,2.0


In [14]:
print(vauxhall_data.info())
display(vauxhall_data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13632 entries, 0 to 13631
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         13632 non-null  object 
 1   year          13632 non-null  int64  
 2   price         13632 non-null  int64  
 3   transmission  13632 non-null  object 
 4   mileage       13632 non-null  int64  
 5   fuelType      13632 non-null  object 
 6   tax           13632 non-null  int64  
 7   mpg           13632 non-null  float64
 8   engineSize    13632 non-null  float64
dtypes: float64(2), int64(4), object(3)
memory usage: 958.6+ KB
None


,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,Corsa,2018,7885,Manual,9876,Petrol,145,55.4,1.4
1,Corsa,2019,11995,Manual,2500,Petrol,145,54.3,1.4
2,Corsa,2017,9777,Automatic,9625,Petrol,145,47.9,1.4
3,Corsa,2016,8500,Manual,25796,Petrol,30,55.4,1.4
4,Corsa,2019,10000,Manual,3887,Petrol,145,43.5,1.4


In [15]:
print(vw_data.info())
display(vw_data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15157 entries, 0 to 15156
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         15157 non-null  object 
 1   year          15157 non-null  int64  
 2   price         15157 non-null  int64  
 3   transmission  15157 non-null  object 
 4   mileage       15157 non-null  int64  
 5   fuelType      15157 non-null  object 
 6   tax           15157 non-null  int64  
 7   mpg           15157 non-null  float64
 8   engineSize    15157 non-null  float64
dtypes: float64(2), int64(4), object(3)
memory usage: 1.0+ MB
None


,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,T-Roc,2019,25000,Automatic,13904,Diesel,145,49.6,2.0
1,T-Roc,2019,26883,Automatic,4562,Diesel,145,49.6,2.0
2,T-Roc,2019,20000,Manual,7414,Diesel,145,50.4,2.0
3,T-Roc,2019,33492,Automatic,4825,Petrol,145,32.5,2.0
4,T-Roc,2019,22900,Semi-Auto,6500,Petrol,150,39.8,1.5


In [16]:
audi_data['model'] = 'audi'
bmw_data['model'] = 'bmw'
ford_data['model'] = 'ford'
hyundi_data['model'] = 'hyundai'
merc_data['model'] = 'mercedez'
skoda_data['model'] = 'skoda'
toyota_data['model'] = 'toyota'
vauxhall_data['model'] = 'vauxhall'
vw_data['model'] = 'vw'

In [17]:
used_cars=pd.concat([audi_data,bmw_data,ford_data,hyundi_data,merc_data,skoda_data,toyota_data,vauxhall_data,vw_data])

In [18]:
used_cars.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99187 entries, 0 to 15156
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         99187 non-null  object 
 1   year          99187 non-null  int64  
 2   price         99187 non-null  int64  
 3   transmission  99187 non-null  object 
 4   mileage       99187 non-null  int64  
 5   fuelType      99187 non-null  object 
 6   tax           99187 non-null  int64  
 7   mpg           99187 non-null  float64
 8   engineSize    99187 non-null  float64
dtypes: float64(2), int64(4), object(3)
memory usage: 7.6+ MB


In [19]:
used_cars['model'].value_counts()

ford        17965
vw          15157
vauxhall    13632
mercedez    13119
bmw         10781
audi        10668
toyota       6738
skoda        6267
hyundai      4860
Name: model, dtype: int64

In [20]:
used_cars['year'].value_counts()

2019    26771
2017    21733
2016    15850
2018    13664
2015     7645
2020     4465
2014     4085
2013     2603
2012      634
2011      432
2010      340
2009      278
2008      200
2007      163
2006       86
2005       69
2004       52
2003       34
2002       31
2001       20
2000        9
1998        8
1999        6
1997        4
1970        2
1996        2
2060        1
Name: year, dtype: int64

In [21]:
used_cars['transmission'].value_counts()

Manual       56445
Semi-Auto    22677
Automatic    20056
Other            9
Name: transmission, dtype: int64

In [22]:
used_cars['fuelType'].value_counts()

Petrol      54928
Diesel      40928
Hybrid       3078
Other         247
Electric        6
Name: fuelType, dtype: int64

In [23]:
used_cars.describe().T

,count,mean,std,min,25%,50%,75%,max
year,99187.0,2017.087723,2.123934,1970.0,2016.0,2017.0,2019.0,2060.0
price,99187.0,16805.347656,9866.773417,450.0,9999.0,14495.0,20870.0,159999.0
mileage,99187.0,23058.914213,21148.523721,1.0,7425.0,17460.0,32339.0,323000.0
tax,99187.0,120.299838,63.150926,0.0,125.0,145.0,145.0,580.0
mpg,99187.0,55.166825,16.138522,0.3,47.1,54.3,62.8,470.8
engineSize,99187.0,1.663280,0.557646,0.0,1.2,1.6,2.0,6.6


In [24]:
used_cars = used_cars[used_cars['year'] <= 2021]

In [25]:
used_cars.describe().T

,count,mean,std,min,25%,50%,75%,max
year,99186.0,2017.087291,2.119570,1970.0,2016.0,2017.0,2019.00,2020.0
price,99186.0,16805.451606,9866.768844,450.0,9999.0,14495.0,20870.00,159999.0
mileage,99186.0,23058.594126,21148.390071,1.0,7424.5,17460.0,32337.75,323000.0
tax,99186.0,120.298984,63.150672,0.0,125.0,145.0,145.00,580.0
mpg,99186.0,55.166950,16.138556,0.3,47.1,54.3,62.80,470.8
engineSize,99186.0,1.663283,0.557649,0.0,1.2,1.6,2.00,6.6


In [26]:
used_cars.isna().sum()

model           0
year            0
price           0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
dtype: int64

In [27]:
print(used_cars.shape)

(99186, 9)


In [28]:
used_cars

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,audi,2017,12500,Manual,15735,Petrol,150,55.4,1.4
1,audi,2016,16500,Automatic,36203,Diesel,20,64.2,2.0
2,audi,2016,11000,Manual,29946,Petrol,30,55.4,1.4
3,audi,2017,16800,Automatic,25952,Diesel,145,67.3,2.0
4,audi,2019,17300,Manual,1998,Petrol,145,49.6,1.0
...,...,...,...,...,...,...,...,...,...
15152,vw,2012,5990,Manual,74000,Diesel,125,58.9,2.0
15153,vw,2008,1799,Manual,88102,Petrol,145,46.3,1.2
15154,vw,2009,1590,Manual,70000,Petrol,200,42.0,1.4
15155,vw,2006,1250,Manual,82704,Petrol,150,46.3,1.2


In [29]:
used_cars.isna().sum()

model           0
year            0
price           0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
dtype: int64

In [30]:
used_cars['transmission'].replace(to_replace=['Manual'],value=0,inplace=True)
used_cars['transmission'].replace(to_replace=['Semi-Auto'],value=1,inplace=True)
used_cars['transmission'].replace(to_replace=['Automatic'],value=2,inplace=True)
used_cars['transmission'].replace(to_replace=['Other'],value=3,inplace=True)

In [31]:
used_cars['fuelType'].replace(to_replace=['Petrol'],value=10,inplace=True)
used_cars['fuelType'].replace(to_replace=['Diesel'],value=9,inplace=True)
used_cars['fuelType'].replace(to_replace=['Hybrid'],value=8,inplace=True)
used_cars['fuelType'].replace(to_replace=['Other'],value=7,inplace=True)
used_cars['fuelType'].replace(to_replace=['Electric'],value=6,inplace=True)

In [32]:
used_cars['model'].replace(to_replace=['ford'],value=30,inplace=True)
used_cars['model'].replace(to_replace=['vw'],value=29,inplace=True)
used_cars['model'].replace(to_replace=['vauxhall'],value=28,inplace=True)
used_cars['model'].replace(to_replace=['mercedez'],value=27,inplace=True)
used_cars['model'].replace(to_replace=['bmw'],value=26,inplace=True)
used_cars['model'].replace(to_replace=['audi'],value=25,inplace=True)
used_cars['model'].replace(to_replace=['toyota'],value=24,inplace=True)
used_cars['model'].replace(to_replace=['skoda'],value=23,inplace=True)
used_cars['model'].replace(to_replace=['hyundai'],value=22,inplace=True)

In [33]:
used_cars['transmission'].value_counts()

0    56445
1    22677
2    20055
3        9
Name: transmission, dtype: int64

In [34]:
used_cars['fuelType'].value_counts()

10    54927
9     40928
8      3078
7       247
6         6
Name: fuelType, dtype: int64

In [35]:
used_cars['model'].value_counts()

30    17964
29    15157
28    13632
27    13119
26    10781
25    10668
24     6738
23     6267
22     4860
Name: model, dtype: int64

In [36]:
used_cars

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,25,2017,12500,0,15735,10,150,55.4,1.4
1,25,2016,16500,2,36203,9,20,64.2,2.0
2,25,2016,11000,0,29946,10,30,55.4,1.4
3,25,2017,16800,2,25952,9,145,67.3,2.0
4,25,2019,17300,0,1998,10,145,49.6,1.0
...,...,...,...,...,...,...,...,...,...
15152,29,2012,5990,0,74000,9,125,58.9,2.0
15153,29,2008,1799,0,88102,10,145,46.3,1.2
15154,29,2009,1590,0,70000,10,200,42.0,1.4
15155,29,2006,1250,0,82704,10,150,46.3,1.2


In [37]:
used_cars.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99186 entries, 0 to 15156
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         99186 non-null  int64  
 1   year          99186 non-null  int64  
 2   price         99186 non-null  int64  
 3   transmission  99186 non-null  int64  
 4   mileage       99186 non-null  int64  
 5   fuelType      99186 non-null  int64  
 6   tax           99186 non-null  int64  
 7   mpg           99186 non-null  float64
 8   engineSize    99186 non-null  float64
dtypes: float64(2), int64(7)
memory usage: 7.6 MB


In [38]:
used_cars.describe().T

,count,mean,std,min,25%,50%,75%,max
model,99186.0,26.961073,2.400729,22.0,25.0,27.0,29.00,30.0
year,99186.0,2017.087291,2.119570,1970.0,2016.0,2017.0,2019.00,2020.0
price,99186.0,16805.451606,9866.768844,450.0,9999.0,14495.0,20870.00,159999.0
transmission,99186.0,0.633295,0.798232,0.0,0.0,0.0,1.00,3.0
mileage,99186.0,23058.594126,21148.390071,1.0,7424.5,17460.0,32337.75,323000.0
fuelType,99186.0,9.517583,0.572212,6.0,9.0,10.0,10.00,10.0
tax,99186.0,120.298984,63.150672,0.0,125.0,145.0,145.00,580.0
mpg,99186.0,55.166950,16.138556,0.3,47.1,54.3,62.80,470.8
engineSize,99186.0,1.663283,0.557649,0.0,1.2,1.6,2.00,6.6


In [39]:
x_train,x_test=train_test_split(used_cars,test_size=0.20)

In [40]:
x_train

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
3762,28,2017,11700,0,11886,10,150,51.4,1.4
9129,26,2018,22988,2,21276,9,145,56.5,3.0
9104,28,2017,13625,2,8586,10,145,43.5,1.4
7681,25,2014,11650,0,22014,10,145,48.7,1.8
10454,28,2017,10798,0,23325,10,145,41.5,1.4
...,...,...,...,...,...,...,...,...,...
2717,24,2016,9490,0,19000,10,30,55.0,1.3
5607,23,2019,16980,2,3500,10,150,47.1,1.0
9091,30,2016,14995,0,40136,9,145,54.3,2.0
5308,29,2013,6900,0,72230,9,0,68.9,1.6


In [41]:
x_test

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
1594,22,2018,7250,0,24040,10,145,57.7,1.2
860,23,2020,26995,1,1000,9,145,47.1,2.0
16407,30,2015,8275,0,35000,10,20,62.8,1.0
6076,26,2019,27781,0,11701,9,150,54.3,2.0
104,22,2016,13300,0,25071,9,30,61.7,1.7
...,...,...,...,...,...,...,...,...,...
3455,23,2017,8290,0,29988,10,145,60.1,1.2
5511,27,2019,19182,1,7807,9,145,65.7,1.5
1043,28,2016,8398,0,12815,10,30,55.4,1.4
3072,30,2019,15910,0,5752,10,145,58.9,1.0


In [42]:
col_names=list(used_cars.columns[used_cars.columns!="price"])

In [43]:
col_names

['model',
 'year',
 'transmission',
 'mileage',
 'fuelType',
 'tax',
 'mpg',
 'engineSize']

In [44]:
scaler = StandardScaler()
scaled_data=scaler.fit_transform(x_train.loc[:,x_train.columns!="price"])

In [45]:
x_train[col_names]=scaled_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [46]:
scaler = StandardScaler()
scaled_data=scaler.fit_transform(x_test.loc[:,x_test.columns!="price"])

In [47]:
x_test[col_names]=scaled_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [48]:
x_train

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
3762,0.435362,-0.038786,11700,-0.795564,-0.527537,0.844924,0.468402,-0.231241,-0.475442
9129,-0.398839,0.431147,22988,1.708459,-0.085431,-0.906324,0.389379,0.086705,2.386756
9104,0.435362,-0.038786,13625,1.708459,-0.682910,0.844924,0.389379,-0.723746,-0.475442
7681,-0.815940,-1.448583,11650,-0.795564,-0.050684,0.844924,0.389379,-0.399565,0.240107
10454,0.435362,-0.038786,10798,-0.795564,0.011042,0.844924,0.389379,-0.848431,-0.475442
...,...,...,...,...,...,...,...,...,...
2717,-1.233041,-0.508718,9490,-0.795564,-0.192591,0.844924,-1.428142,-0.006808,-0.654329
5607,-1.650141,0.901079,16980,1.708459,-0.922372,0.844924,0.468402,-0.499313,-1.190991
9091,1.269563,-0.508718,14995,-0.795564,0.802549,-0.906324,0.389379,-0.050448,0.597882
5308,0.852463,-1.918516,6900,-0.795564,2.313620,-0.906324,-1.902278,0.859751,-0.117667


In [49]:
x_test

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
1594,-2.063656,0.428527,7250,-0.784655,0.053366,0.835802,0.398303,0.139328,-0.821151
860,-1.649308,1.387534,26995,0.471274,-1.055391,-0.897487,0.398303,-0.502212,0.628024
16407,1.251126,-1.009984,8275,-0.784655,0.580795,0.835802,-1.596691,0.447994,-1.183445
6076,-0.406265,0.908031,27781,-0.784655,-0.540426,-0.897487,0.478103,-0.066449,0.628024
104,-2.063656,-0.530480,13300,-0.784655,0.102981,-0.897487,-1.437092,0.381419,0.084583
...,...,...,...,...,...,...,...,...,...
3455,-1.649308,-0.050977,8290,-0.784655,0.339602,0.835802,0.398303,0.284583,-0.821151
5511,0.008083,0.908031,19182,0.471274,-0.727817,-0.897487,0.398303,0.623510,-0.277710
1043,0.422431,-0.530480,8398,-0.784655,-0.486817,0.835802,-1.437092,0.000126,-0.458857
3072,1.251126,0.908031,15910,-0.784655,-0.826710,0.835802,0.398303,0.211956,-1.183445


## Baseline for comparison...

In [ ]:
#('C', 1000.0), ('degree', 2), ('epsilon', 100.0), ('kernel', 'rbf')
start = time.time()
base_rbf=SVR(kernel='rbf',C=1000.0,epsilon=100.0).fit(x_train.loc[:,x_train.columns!="price"], x_train.loc[:,"price"])
end = time.time()
pred_rbf = base_rbf.predict(x_test.loc[:,x_train.columns!="price"])
# mean_absolute_error(y_true, y_pred)
# mean_squared_error(y_true, y_pred)
print("MAE: ",mean_absolute_error(x_test.loc[:,"price"], pred_rbf))
print("RMSE: ",math.sqrt(mean_squared_error(x_test.loc[:,"price"], pred_rbf)))
print("SCORE: ",base_rbf.score(x_test.loc[:,x_train.columns!="price"], x_test.loc[:,"price"]))
print(str("total time: ")+ (str(end - start)))

MAE:  2209.2865385832447
RMSE:  3546.0862417132466
SCORE:  0.8687557919363377
total time: 505.31611728668213


In [ ]:
#('C', 985.3187353064709), ('degree', 5), ('epsilon', 71.84431098910146), ('kernel', 'linear')
start = time.time()
base_linear=SVR(kernel='linear',C= 985.3187353064709,epsilon=71.84431098910146).fit(x_train.loc[:,x_train.columns!="price"], x_train.loc[:,"price"])
end = time.time()
pred_linear = base_linear.predict(x_test.loc[:,x_train.columns!="price"])
# mean_absolute_error(y_true, y_pred)
# mean_squared_error(y_true, y_pred)
print("MAE: ",mean_absolute_error(x_test.loc[:,"price"], pred_linear))
print("RMSE: ",math.sqrt(mean_squared_error(x_test.loc[:,"price"], pred_linear)))
print("SCORE: ",base_linear.score(x_test.loc[:,x_train.columns!="price"], x_test.loc[:,"price"]))
print(str("total time: ")+ (str(end - start)))

MAE:  3384.9388681724736
RMSE:  5336.00896310862
SCORE:  0.7028233652650696
total time: 502.5020453929901


In [ ]:
#('C', 134.41572295100173), ('degree', 3), ('epsilon', 100.0), ('kernel', 'poly')
start = time.time()
base_poly=SVR(kernel='poly',C=134.41572295100173,degree=3,epsilon=100.0).fit(x_train.loc[:,x_train.columns!="price"], x_train.loc[:,"price"])
end = time.time()
pred_poly = base_poly.predict(x_test.loc[:,x_train.columns!="price"])
# mean_absolute_error(y_true, y_pred)
# mean_squared_error(y_true, y_pred)
print("MAE: ",mean_absolute_error(x_test.loc[:,"price"], pred_poly))
print("RMSE: ",math.sqrt(mean_squared_error(x_test.loc[:,"price"], pred_poly)))
print("SCORE: ",base_poly.score(x_test.loc[:,x_train.columns!="price"], x_test.loc[:,"price"]))
print(str("total time: ")+ (str(end - start)))

MAE:  3676.8604371181164
RMSE:  6108.838744507165
SCORE:  0.6105076950325452
total time: 435.56417989730835


In [ ]:
#('C', 38.811563428968796), ('degree', 4), ('epsilon', 0.014594759519076954), ('kernel', 'sigmoid')]
start = time.time()
base_sig=SVR(kernel='sigmoid',C=38.811563428968796,epsilon=0.014594759519076954, degree=4).fit(x_train.loc[:,x_train.columns!="price"], x_train.loc[:,"price"])
end = time.time()
pred_sig = base_sig.predict(x_test.loc[:,x_train.columns!="price"])
# mean_absolute_error(y_true, y_pred)
# mean_squared_error(y_true, y_pred)
print("MAE: ",mean_absolute_error(x_test.loc[:,"price"], pred_sig))
print("RMSE: ",math.sqrt(mean_squared_error(x_test.loc[:,"price"], pred_sig)))
print("SCORE: ",base_sig.score(x_test.loc[:,x_train.columns!="price"], x_test.loc[:,"price"]))
print(str("total time: ")+ (str(end - start)))

MAE:  18663.669417927966
RMSE:  28128.38167250649
SCORE:  -7.257927244822845
total time: 2958.1631731987


## SVR subsmple with Bayesian Optimization

In [ ]:
svr_subsample_bayes_train(x_train,x_test,0.01,0.1,"price","rbf",5)

####### BAYESIAN_OPT MODE #######
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    2.2s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    8.8s finished
/usr/local/lib/python3.7/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    8.9s finished


++++++ TRAIN TIME (1) 507.8170509338379 +++++++++++
####### BEST MODEL PARAMS STEP 1 #####
OrderedDict([('C', 1000.0), ('degree', 2), ('epsilon', 100.0), ('kernel', 'rbf')])
Coefficient of determination R^2 of the prediction: 

0.806311190619588
MAE:  2482.7738780265836
RMSE:  4307.859049180302
DIFF with previous:  -0.10785235976265006
####### BEST MODEL PARAMS STEP 1 #####
!!!! TOTAL ELAPSED TIME: 595.1942837238312 !!!!


BayesSearchCV(cv=3, error_score='raise',
              estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                            epsilon=0.1, gamma='scale', kernel='rbf',
                            max_iter=-1, shrinking=True, tol=0.001,
                            verbose=False),
              fit_params=None, iid=True, n_iter=50, n_jobs=3, n_points=1,
              optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
              refit=True, return_train_score=False, scoring=None,
              search_spaces={'C': (0.01, 1000.0, 'log-uniform'),
                             'degree': (2, 5),
                             'epsilon': (0.01, 100.0, 'log-uniform'),
                             'kernel': ['rbf']},
              verbose=3)

In [ ]:
svr_subsample_bayes_train(x_train,x_test,0.01,0.1,"price","linear",5)

####### BAYESIAN_OPT MODE #######
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    1.8s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    5.2s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    7.1s finished


++++++ TRAIN TIME (1) 385.50993299484253 +++++++++++
####### BEST MODEL PARAMS STEP 1 #####
OrderedDict([('C', 985.3187353064709), ('degree', 5), ('epsilon', 71.84431098910146), ('kernel', 'linear')])
Coefficient of determination R^2 of the prediction: 

0.7082612580254348
MAE:  3394.6375221094763
RMSE:  5286.963025330365
DIFF with previous:  0.003243295030890714
####### BEST MODEL PARAMS STEP 1 #####
!!!! TOTAL ELAPSED TIME: 461.8501751422882 !!!!


BayesSearchCV(cv=3, error_score='raise',
              estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                            epsilon=0.1, gamma='scale', kernel='rbf',
                            max_iter=-1, shrinking=True, tol=0.001,
                            verbose=False),
              fit_params=None, iid=True, n_iter=50, n_jobs=3, n_points=1,
              optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
              refit=True, return_train_score=False, scoring=None,
              search_spaces={'C': (0.01, 1000.0, 'log-uniform'),
                             'degree': (2, 5),
                             'epsilon': (0.01, 100.0, 'log-uniform'),
                             'kernel': ['linear']},
              verbose=3)

In [ ]:
svr_subsample_bayes_train(x_train,x_test,0.01,0.1,"price","poly",5)

####### BAYESIAN_OPT MODE #######
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    1.9s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    6.6s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    6.8s finished


++++++ TRAIN TIME (1) 528.7366309165955 +++++++++++
####### BEST MODEL PARAMS STEP 1 #####
OrderedDict([('C', 134.41572295100173), ('degree', 3), ('epsilon', 100.0), ('kernel', 'poly')])
Coefficient of determination R^2 of the prediction: 

0.5674757424430397
MAE:  3911.1544304577274
RMSE:  6437.458686181871
DIFF with previous:  -0.6299884663629429
####### BEST MODEL PARAMS STEP 1 #####
!!!! TOTAL ELAPSED TIME: 654.255047082901 !!!!


BayesSearchCV(cv=3, error_score='raise',
              estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                            epsilon=0.1, gamma='scale', kernel='rbf',
                            max_iter=-1, shrinking=True, tol=0.001,
                            verbose=False),
              fit_params=None, iid=True, n_iter=50, n_jobs=3, n_points=1,
              optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
              refit=True, return_train_score=False, scoring=None,
              search_spaces={'C': (0.01, 1000.0, 'log-uniform'),
                             'degree': (2, 5),
                             'epsilon': (0.01, 100.0, 'log-uniform'),
                             'kernel': ['poly']},
              verbose=3)

In [ ]:
svr_subsample_bayes_train(x_train,x_test,0.01,0.1,"price","sigmoid",5)

####### BAYESIAN_OPT MODE #######
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    1.9s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.1s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:   10.4s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:   10.6s finished


++++++ TRAIN TIME (1) 667.3490178585052 +++++++++++
####### BEST MODEL PARAMS STEP 1 #####
OrderedDict([('C', 38.811563428968796), ('degree', 4), ('epsilon', 0.014594759519076954), ('kernel', 'sigmoid')])
Coefficient of determination R^2 of the prediction: 

0.5896111053513677
MAE:  3880.888497037091
RMSE:  6270.5699981232865
DIFF with previous:  0.04213886951285328
####### BEST MODEL PARAMS STEP 1 #####
!!!! TOTAL ELAPSED TIME: 762.7191827297211 !!!!


BayesSearchCV(cv=3, error_score='raise',
              estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                            epsilon=0.1, gamma='scale', kernel='rbf',
                            max_iter=-1, shrinking=True, tol=0.001,
                            verbose=False),
              fit_params=None, iid=True, n_iter=50, n_jobs=3, n_points=1,
              optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
              refit=True, return_train_score=False, scoring=None,
              search_spaces={'C': (0.01, 1000.0, 'log-uniform'),
                             'degree': (2, 5),
                             'epsilon': (0.01, 100.0, 'log-uniform'),
                             'kernel': ['sigmoid']},
              verbose=3)

## Comparison without the subsample algorithm

In [ ]:
import math
start = time.time()
Sv = BayesSearchCV(
    SVR(gamma='scale'),
    {
        'C': (1e-2, 1e+3,'log-uniform'),
        'degree': (2, 5),
        'epsilon': (1e-2, 1e+2,'log-uniform'),  # integer valued parameter
        'kernel': ['rbf']  # categorical parameter
    },
    cv=3,
    n_jobs=3,
    verbose=3
)

Sv.fit(x_train.loc[:,x_train.columns!="price"], x_train.loc[:,"price"])
end = time.time()
pred = Sv.predict(x_test.loc[:,x_train.columns!="price"])
# mean_absolute_error(y_true, y_pred)
# mean_squared_error(y_true, y_pred)
print("MAE: ",mean_absolute_error(x_test.loc[:,"price"], pred))
print("RMSE: ",math.sqrt(mean_squared_error(x_test.loc[:,"price"], pred)))
print("SCORE: ",Sv.score(x_test.loc[:,x_train.columns!="price"], x_test.loc[:,"price"]))
print(str("total time: ")+ (str(end - start)))
print("BEST PARAMS:"+str(Sv.best_params_))

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed: 10.0min finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  9.9min finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  9.1min finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  9.0min finished


MAE:  2242.250591245884
RMSE:  3719.6801227598908
SCORE:  0.8596166965501961
total time: 28116.238686323166
BEST PARAMS:OrderedDict([('C', 1000.0), ('degree', 5), ('epsilon', 100.0), ('kernel', 'rbf')])


In [ ]:
import math
start = time.time()
Sv = BayesSearchCV(
    SVR(gamma='scale'),
    {
        'C': (1e-2, 1e+3,'log-uniform'),
        'degree': (2, 5),
        'epsilon': (1e-2, 1e+2,'log-uniform'),  # integer valued parameter
        'kernel': ['linear']  # categorical parameter
    },
    cv=3,
    n_jobs=3,
    verbose=3
)

Sv.fit(x_train.loc[:,x_train.columns!="price"], x_train.loc[:,"price"])
end = time.time()
pred = Sv.predict(x_test.loc[:,x_train.columns!="price"])
# mean_absolute_error(y_true, y_pred)
# mean_squared_error(y_true, y_pred)
print("MAE: ",mean_absolute_error(x_test.loc[:,"price"], pred))
print("RMSE: ",math.sqrt(mean_squared_error(x_test.loc[:,"price"], pred)))
print("SCORE: ",Sv.score(x_test.loc[:,x_train.columns!="price"], x_test.loc[:,"price"]))
print(str("total time: ")+ (str(end - start)))
print("BEST PARAMS:"+str(Sv.best_params_))

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  5.4min finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  5.4min finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  7.2min finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  5.6min finished


MAE:  3394.2510794359023
RMSE:  5449.762621377103
SCORE:  0.6986581558790688
total time: 18549.290234565735
BEST PARAMS:OrderedDict([('C', 982.0764930793201), ('degree', 5), ('epsilon', 93.59911894118734), ('kernel', 'linear')])


In [ ]:
import math
start = time.time()
Sv = BayesSearchCV(
    SVR(gamma='scale'),
    {
        'C': (1e-2, 1e+3,'log-uniform'),
        'degree': (2, 5),
        'epsilon': (1e-2, 1e+2,'log-uniform'),  # integer valued parameter
        'kernel': ['poly']  # categorical parameter
    },
    cv=3,
    n_jobs=3,
    verbose=3
)

Sv.fit(x_train.loc[:,x_train.columns!="price"], x_train.loc[:,"price"])
end = time.time()
pred = Sv.predict(x_test.loc[:,x_train.columns!="price"])
# mean_absolute_error(y_true, y_pred)
# mean_squared_error(y_true, y_pred)
print("MAE: ",mean_absolute_error(x_test.loc[:,"price"], pred))
print("RMSE: ",math.sqrt(mean_squared_error(x_test.loc[:,"price"], pred)))
print("SCORE: ",Sv.score(x_test.loc[:,x_train.columns!="price"], x_test.loc[:,"price"]))
print(str("total time: ")+ (str(end - start)))
print("BEST PARAMS:"+str(Sv.best_params_))

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  7.4min finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed: 184.2min finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  9.0min finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  7.1min finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  7.1min finished


MAE:  3777.3756637432884
RMSE:  6088.4276386987685
SCORE:  0.623890271091997
total time: 32871.36774969101
BEST PARAMS:OrderedDict([('C', 42.924497345675924), ('degree', 3), ('epsilon', 100.0), ('kernel', 'poly')])


In [ ]:
import math
start = time.time()
Sv = BayesSearchCV(
    SVR(gamma='scale'),
    {
        'C': (1e-2, 1e+3,'log-uniform'),
        'degree': (2, 5),
        'epsilon': (1e-2, 1e+2,'log-uniform'),  # integer valued parameter
        'kernel': ['sigmoid']  # categorical parameter
    },
    cv=3,
    n_jobs=3,
    verbose=3
)

Sv.fit(x_train.loc[:,x_train.columns!="price"], x_train.loc[:,"price"])
end = time.time()
pred = Sv.predict(x_test.loc[:,x_train.columns!="price"])
# mean_absolute_error(y_true, y_pred)
# mean_squared_error(y_true, y_pred)
print("MAE: ",mean_absolute_error(x_test.loc[:,"price"], pred))
print("RMSE: ",math.sqrt(mean_squared_error(x_test.loc[:,"price"], pred)))
print("SCORE: ",Sv.score(x_test.loc[:,x_train.columns!="price"], x_test.loc[:,"price"]))
print(str("total time: ")+ (str(end - start)))
print("BEST PARAMS:"+str(Sv.best_params_))

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed: 43.6min finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed: 11.8min finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed: 12.6min finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed: 20.7min finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
